In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
df = pd.read_pickle('pickle/df.pkl')

In [6]:
# plt.plot(df[df['state_name'] == 'China']['year'], df[df['state_name'] == 'China']['export_import_ratio'])

In [7]:
# df[df['export_import_ratio'].isnull() == False]['state_name'].value_counts()

In [8]:
df.head()

,cow_code,state_name,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,avg_hostility_level,revision_type_mode,outcome_mode,settlement_type_mode,majority_religion,un_region,un_continent,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_to_democracy,transition_to_dictatorship,age_govt,num_transitions_ever
0,2,United States of America,1946,65.00,160000000.00,14.40,45133984000.00,3030000.00,120842000000.00,2376288000000.00,141389000.00,39725000.00,0.36,19.00,0.00,0.00,19.00,0.00,0.00,19.00,2.00,1.00,1.50,0.27,0.00,0.50,0.00,0.00,3.50,N/A,Stalemate,None Value,Christianity,Northern America,Americas,Direct election,Elective legislature,Elected,Multiple parties legally allowed,Multiple parties,Multiple parties,Legislature with multiple parties,Democratic incumbent,No,0.00,2.00,Yes,No,No,No,No,Yes,No,Yes,Presidential democracy,No,No,77.00,0.00
1,20,Canada,1946,65.00,nan,nan,1552759000.00,46000.00,4222000000.00,119264000000.00,12292000.00,2903000.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,1.00,0,NaN,NaN,Christianity,Northern America,Americas,Indirect election,Elective legislature,Elected,Multiple parties legally allowed,Multiple parties,Multiple parties,Legislature with multiple parties,Democratic incumbent,No,0.00,12.00,No,No,No,No,No,Yes,Yes,No,Parliamentary democracy,No,No,27.00,0.00
2,155,Chile,1946,65.00,nan,nan,56849000.00,34000.00,42000000.00,4174000000.00,5640000.00,910000.00,0.00,19.00,0.00,0.00,19.00,0.00,0.00,19.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,1.00,0,NaN,NaN,Christianity,South America,Americas,Direct election,Elective legislature,Elected,Multiple parties legally allowed,Multiple parties,Multiple parties,Legislature with multiple parties,Democratic incumbent,No,3.00,1.00,No,No,No,No,No,Yes,No,Yes,Presidential democracy,No,No,15.00,1.00
3,640,Turkey,1946,65.00,nan,nan,278779000.00,689000.00,160000000.00,5494000000.00,19074000.00,1435000.00,0.01,4.00,0.00,0.00,0.00,0.00,4.00,3.00,1.00,1.00,2.00,0.84,0.00,1.00,0.00,0.00,3.00,N/A,Stalemate,None Value,Islam,Western Asia,Asia,Indirect election,Elective legislature,Elected,Multiple parties legally allowed,Multiple parties,Multiple parties,Legislature with multiple parties,Democratic incumbent,No,0.00,9.00,Yes,No,No,No,No,No,No,No,Military dictatorship,No,No,24.00,0.00
4,560,South Africa,1946,65.00,nan,nan,96481000.00,40000.00,1032000000.00,49310000000.00,11451000.00,1839000.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,1.00,0,NaN,NaN,Christianity,Southern Africa,Africa,Indirect election,Elective legislature,Elected,Multiple parties legally allowed,Multiple parties,Multiple parties,Legislature with multiple parties,Democratic incumbent,No,0.00,8.00,No,No,No,No,No,No,No,No,Civilian dictatorship,No,No,1.00,0.00


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9172 entries, 0 to 9171
Data columns (total 59 columns):
cow_code                          9172 non-null int64
state_name                        9172 non-null object
year                              9172 non-null int64
num_trade_states                  8946 non-null float64
export_dollars                    8640 non-null float64
import_dollars                    8626 non-null float64
military_expenditure              8409 non-null float64
military_personnel                8813 non-null float64
iron_steel_prod                   8949 non-null float64
prim_energy_consumption           8898 non-null float64
total_pop                         8950 non-null float64
urban_pop                         8950 non-null float64
cinc_score                        8950 non-null float64
num_alliances                     9172 non-null float64
pre_1816_alliances                9172 non-null float64
num_in_effect_1231_2012           9172 non-null float64
de

In [10]:
# from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler

# df_numeric = df.select_dtypes(include=[np.float]).as_matrix()

# my_imputer = IterativeImputer()
# X_trained_filled = my_imputer.fit_transform(df_numeric)

# now transform test
# X_test_filled = my_imputer.transform(X_test)

# X_filled_ii = IterativeImputer().fit_transform(df_numeric)
# ii_mse = ((X_filled_ii[missing_mask] - X[missing_mask]) ** 2).mean()
# print("Iterative Imputer norm minimization MSE: %f" % ii_mse)

# I now run fancyimpute KNN, 
# it returns a np.array which I store as a pandas dataframe
# df_filled = pd.DataFrame(KNN(3).complete(df_numeric))

# X_incomplete_normalized = BiScaler().fit_transform(df_numeric)
# X_filled_softimpute = SoftImpute().fit_transform(X_incomplete_normalized)

In [11]:
# impute for total non-religious
# do any other religion values not make sense as 0?
# use threshold for first non-zero
# maybe a relevant 0 is any 0 followed by a number greater than 10,000?

In [12]:
len(df.columns)

59

In [13]:
df.columns

Index(['cow_code', 'state_name', 'year', 'num_trade_states', 'export_dollars',
       'import_dollars', 'military_expenditure', 'military_personnel',
       'iron_steel_prod', 'prim_energy_consumption', 'total_pop', 'urban_pop',
       'cinc_score', 'num_alliances', 'pre_1816_alliances',
       'num_in_effect_1231_2012', 'defense_treaties', 'neutrality_treaties',
       'nonaggression_treaties', 'entente_treaties', 'num_conflicts',
       'avg_with', 'avg_against', 'avg_cum_duration', 'num_wars',
       'origin_participant_pct', 'ongoing_2010', 'revision_pct',
       'avg_hostility_level', 'revision_type_mode', 'outcome_mode',
       'settlement_type_mode', 'majority_religion', 'un_region',
       'un_continent', 'election_type', 'legislation_type',
       'legislature_status', 'party_legal_status', 'party_existance',
       'party_existance_outside_regime', 'legislature_parties',
       'incumbent_type', 'collective_leadership', 'num_leadership_changes',
       'leader_tenure', 'milit

In [14]:
df.drop(4018, inplace = True)
df.drop(6615, inplace = True)
df.drop(6787, inplace = True)
df.drop(6938, inplace = True)
df.drop(7813, inplace = True)
df.drop(8797, inplace = True)
df.drop(9092, inplace = True)
df.drop(8892, inplace = True)
df.drop(8922, inplace = True)
df.drop(8850, inplace = True)
df.drop(9129, inplace = True)
df.drop(9048, inplace = True)
df.drop(9155, inplace = True)

In [15]:
for year in df['year'].unique():
    if df[df['year'] == year]['cow_code'].value_counts().values[0] > 1:
        print(year)

In [16]:
df.loc[df['num_trade_states'].isnull(), 'num_trade_states'] = 0
df.loc[df['cinc_score'].isnull(), 'cinc_score'] = 0
df.loc[df['export_dollars'].isnull(), 'export_dollars'] = 0
df.loc[df['import_dollars'].isnull(), 'import_dollars'] = 0
df.loc[df['military_expenditure'].isnull(), 'military_expenditure'] = 0
df.loc[df['military_personnel'].isnull(), 'military_personnel'] = 0
df.loc[df['iron_steel_prod'].isnull(), 'iron_steel_prod'] = 0
df.loc[df['prim_energy_consumption'].isnull(), 'prim_energy_consumption'] = 0
df.loc[df['total_pop'].isnull(), 'total_pop'] = 0
df.loc[df['urban_pop'].isnull(), 'urban_pop'] = 0

df.loc[[np.isnan(s) for s in df['num_trade_states']], 'num_trade_states'] = 0
df.loc[[np.isnan(s) for s in df['cinc_score']], 'cinc_score'] = 0
df.loc[[np.isnan(s) for s in df['export_dollars']], 'export_dollars'] = 0
df.loc[[np.isnan(s) for s in df['import_dollars']], 'import_dollars'] = 0
df.loc[[np.isnan(s) for s in df['military_expenditure']], 'military_expenditure'] = 0
df.loc[[np.isnan(s) for s in df['military_personnel']], 'military_personnel'] = 0
df.loc[[np.isnan(s) for s in df['iron_steel_prod']], 'iron_steel_prod'] = 0
df.loc[[np.isnan(s) for s in df['prim_energy_consumption']], 'prim_energy_consumption'] = 0
df.loc[[np.isnan(s) for s in df['total_pop']], 'total_pop'] = 0
df.loc[[np.isnan(s) for s in df['urban_pop']], 'urban_pop'] = 0

df.loc[df['export_dollars'] < 1000, 'export_dollars'] = 0
df.loc[df['import_dollars'] < 1000, 'import_dollars'] = 0

In [17]:
df.drop(['iron_steel_prod'], axis = 1, inplace = True)

In [18]:
print('total possible 0:\t', len(df))
print('num_trade_states == 0:\t', len(df[df['num_trade_states'] == 0]), '\t', pd.Series((len(df[df['num_trade_states'] == 0])/len(df))).values[0].round(3))
print('cinc_score == 0:\t', len(df[df['cinc_score'] == 0]), '\t', pd.Series((len(df[df['cinc_score'] == 0])/len(df))).values[0].round(3))
print('military_personnel == 0:\t', len(df[df['military_personnel'] == 0]), '\t', pd.Series((len(df[df['military_personnel'] == 0])/len(df))).values[0].round(3))
print('prim_energy_consumption == 0:\t', len(df[df['prim_energy_consumption'] == 0]), '\t', pd.Series((len(df[df['prim_energy_consumption'] == 0])/len(df))).values[0].round(3))
print('total_pop == 0:\t', len(df[df['total_pop'] == 0]), '\t', pd.Series((len(df[df['total_pop'] == 0])/len(df))).values[0].round(3))
print('urban_pop == 0:\t', len(df[df['urban_pop'] == 0]), '\t', pd.Series((len(df[df['urban_pop'] == 0])/len(df))).values[0].round(3))

# print('export_dollars == 0:\t', len(df[df['export_dollars'] == 0]), '\t', pd.Series((len(df[df['export_dollars'] == 0])/len(df))).values[0].round(3))
# print('import_dollars == 0:\t', len(df[df['import_dollars'] == 0]), '\t', pd.Series((len(df[df['import_dollars'] == 0])/len(df))).values[0].round(3))
# print('military_expenditure == 0:\t', len(df[df['military_expenditure'] == 0]), '\t', pd.Series((len(df[df['military_expenditure'] == 0])/len(df))).values[0].round(3))

total possible 0:	 9159
num_trade_states == 0:	 226 	 0.025
cinc_score == 0:	 222 	 0.024
military_personnel == 0:	 1108 	 0.121
prim_energy_consumption == 0:	 345 	 0.038
total_pop == 0:	 222 	 0.024
urban_pop == 0:	 1665 	 0.182


In [19]:
# Exponential slopes for export_dollars, import_dollars, military_expenditure

In [20]:
# plt.plot(df[df['state_name'] == 'Netherlands']['year'], df[df['state_name'] == 'Netherlands']['military_personnel'])

In [ ]:
column_list = ['num_trade_states', 'prim_energy_consumption',
               'total_pop', 'urban_pop', 'cinc_score', 'military_personnel']
fix_count = 0

for c_code in df['cow_code'].unique():
    years = list(df[df['cow_code'] == c_code]['year'].unique())
    for column in column_list:
        data_list = []
        for year in years:
            data_list.append(df[(df['cow_code'] == c_code) & (df['year'] == year)][column].values[0])
        for i, item in enumerate(data_list[1:]):
            i+=1
            if item == 0 and data_list.count(0) == len(data_list):
                pass
            elif item == 0 and data_list.count(0) == (len(data_list) - 1):
                pass
            elif item == 0 and data_list[i-1] != 0:
                if i+1 <= (len(data_list) - 1) and data_list[i+1] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+1])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/2)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+2 <= (len(data_list) - 1) and data_list[i+2] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+2])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/3)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+3 <= (len(data_list) - 1) and data_list[i+3] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+3])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/4)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+4 <= (len(data_list) - 1) and data_list[i+4] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/5)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+5 <= (len(data_list) - 1) and data_list[i+5] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/6)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+6 <= (len(data_list) - 1) and data_list[i+6] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/7)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+7 <= (len(data_list) - 1) and data_list[i+7] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/8)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+8 <= (len(data_list) - 1) and data_list[i+8] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/9)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+9 <= (len(data_list) - 1) and data_list[i+9] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/10)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+10 <= (len(data_list) - 1) and data_list[i+10] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/11)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+11 <= (len(data_list) - 1) and data_list[i+11] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/12)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+12 <= (len(data_list) - 1) and data_list[i+12] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/13)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+13 <= (len(data_list) - 1) and data_list[i+13] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/14)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+14 <= (len(data_list) - 1) and data_list[i+14] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/15)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+15 <= (len(data_list) - 1) and data_list[i+15] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/16)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+16 <= (len(data_list) - 1) and data_list[i+16] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/17)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+17 <= (len(data_list) - 1) and data_list[i+17] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/18)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+18 <= (len(data_list) - 1) and data_list[i+18] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/19)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+19 <= (len(data_list) - 1) and data_list[i+19] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/20)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+20 <= (len(data_list) - 1) and data_list[i+20] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/21)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+21 <= (len(data_list) - 1) and data_list[i+21] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/22)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+22 <= (len(data_list) - 1) and data_list[i+22] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/23)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+23 <= (len(data_list) - 1) and data_list[i+23] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/24)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+22]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+24 <= (len(data_list) - 1) and data_list[i+24] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/25)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+22]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+23]), column] = (24 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                elif i+25 <= (len(data_list) - 1) and data_list[i+25] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])/26)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+22]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+23]), column] = (24 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+24]), column] = (25 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i-1])][column].values[0])
                else:
                    print(c_code, years[i], column)
                    fix_count+=1
                    
print(fix_count)

315 1992 prim_energy_consumption
95 2000 military_personnel
395 2002 urban_pop
94 2000 military_personnel
760 2001 military_personnel
817 1975 prim_energy_consumption
352 2002 urban_pop
520 2001 military_personnel


In [ ]:
fix_count = 0

for c_code in df['cow_code'].unique():
    years = list(df[df['cow_code'] == c_code]['year'].unique())
    for column in column_list:
        data_list = []
        for year in years:
            data_list.append(df[(df['cow_code'] == c_code) & (df['year'] == year)][column].values[0])
        for i, item in enumerate(data_list[1:]):
            i+=1
            if item == 0 and data_list.count(0) == len(data_list):
                pass
            elif item == 0 and data_list.count(0) == (len(data_list) - 1):
                pass
            elif item == 0 and data_list[i-1] != 0:
                if data_list[-1] == 0 and data_list[-2] != 0 and data_list[-3] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-2])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-3])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-2])][column].values[0])
                elif data_list[-1] == 0 and data_list[-3] != 0 and data_list[-4] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-3])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-4])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-3])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-3])][column].values[0])
                elif data_list[-1] == 0 and data_list[-4] != 0 and data_list[-5] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-4])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-5])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-4])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-4])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-4])][column].values[0])
                elif data_list[-1] == 0 and data_list[-5] != 0 and data_list[-6] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-5])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-5])][column].values[0])
                elif data_list[-1] == 0 and data_list[-6] != 0 and data_list[-7] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-6])][column].values[0])
                elif data_list[-1] == 0 and data_list[-7] != 0 and data_list[-8] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-7])][column].values[0])
                elif data_list[-1] == 0 and data_list[-8] != 0 and data_list[-9] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-8])][column].values[0])
                elif data_list[-1] == 0 and data_list[-9] != 0 and data_list[-10] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-9])][column].values[0])
                elif data_list[-1] == 0 and data_list[-10] != 0 and data_list[-11] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-10])][column].values[0])
                elif data_list[-1] == 0 and data_list[-11] != 0 and data_list[-12] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-11])][column].values[0])
                elif data_list[-1] == 0 and data_list[-12] != 0 and data_list[-13] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-12])][column].values[0])
                elif data_list[-1] == 0 and data_list[-13] != 0 and data_list[-14] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-13])][column].values[0])
                elif data_list[-1] == 0 and data_list[-14] != 0 and data_list[-15] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-14])][column].values[0])
                elif data_list[-1] == 0 and data_list[-15] != 0 and data_list[-16] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-15])][column].values[0])
                elif data_list[-1] == 0 and data_list[-16] != 0 and data_list[-17] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-16])][column].values[0])
                elif data_list[-1] == 0 and data_list[-17] != 0 and data_list[-18] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-17])][column].values[0])
                elif data_list[-1] == 0 and data_list[-18] != 0 and data_list[-19] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-18])][column].values[0])
                elif data_list[-1] == 0 and data_list[-19] != 0 and data_list[-20] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-19])][column].values[0])
                elif data_list[-1] == 0 and data_list[-20] != 0 and data_list[-21] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-19]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-20])][column].values[0])
                elif data_list[-1] == 0 and data_list[-21] != 0 and data_list[-22] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-20]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-19]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-21])][column].values[0])
                elif data_list[-1] == 0 and data_list[-22] != 0 and data_list[-23] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-21]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-20]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-19]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-22])][column].values[0])
                elif data_list[-1] == 0 and data_list[-23] != 0 and data_list[-24] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-22]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-21]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-20]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-19]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-23])][column].values[0])
                elif data_list[-1] == 0 and data_list[-24] != 0 and data_list[-25] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-23]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-22]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-21]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-20]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-19]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-24])][column].values[0])
                elif data_list[-1] == 0 and data_list[-25] != 0 and data_list[-26] != 0:
                    rate_of_change = ((df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[-26])][column].values[0]))
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-24]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-23]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-22]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-21]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-20]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-19]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-18]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-17]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-16]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-15]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-14]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-13]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-12]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-11]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-10]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-9]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-8]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-7]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-6]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-5]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-4]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-3]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-2]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[-1]), column] = (24 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[-25])][column].values[0])
                else:
                    print(c_code, years[i], column)
                    fix_count+=1

print(fix_count)

In [ ]:
fix_count = 0

for c_code in df['cow_code'].unique():
    years = list(df[df['cow_code'] == c_code]['year'].unique())
    for column in column_list:
        data_list = []
        for year in years:
            data_list.append(df[(df['cow_code'] == c_code) & (df['year'] == year)][column].values[0])
        for i, item in enumerate(data_list):
            if item == 0 and data_list.count(0) == len(data_list):
                pass
            elif item == 0 and data_list.count(0) == (len(data_list) - 1):
                pass
            elif item == 0 and i == 0:
                if i+2 <= (len(data_list) - 1) and data_list[i+1] != 0 and data_list[i+2] != 0:
                    rate_of_change = -1 * ((df[(df['cow_code'] == c_code) & (df['year'] == years[i+2])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+1])][column].values[0])/2)
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+1])][column].values[0])
                elif i+3 <= (len(data_list) - 1) and data_list[i+2] != 0 and data_list[i+3] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+3])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+2])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+2])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+2])][column].values[0])
                elif i+4 <= (len(data_list) - 1) and data_list[i+3] != 0 and data_list[i+4] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+3])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+3])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+3])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+3])][column].values[0])
                elif i+5 <= (len(data_list) - 1) and data_list[i+4] != 0 and data_list[i+5] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+4])][column].values[0])
                elif i+6 <= (len(data_list) - 1) and data_list[i+5] != 0 and data_list[i+6] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+5])][column].values[0])
                elif i+7 <= (len(data_list) - 1) and data_list[i+6] != 0 and data_list[i+7] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+6])][column].values[0])
                elif i+8 <= (len(data_list) - 1) and data_list[i+7] != 0 and data_list[i+8] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+7])][column].values[0])
                elif i+9 <= (len(data_list) - 1) and data_list[i+8] != 0 and data_list[i+9] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+8])][column].values[0])
                elif i+10 <= (len(data_list) - 1) and data_list[i+9] != 0 and data_list[i+10] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+9])][column].values[0])
                elif i+11 <= (len(data_list) - 1) and data_list[i+10] != 0 and data_list[i+11] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+10])][column].values[0])
                elif i+12 <= (len(data_list) - 1) and data_list[i+11] != 0 and data_list[i+12] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+11])][column].values[0])
                elif i+13 <= (len(data_list) - 1) and data_list[i+12] != 0 and data_list[i+13] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+12])][column].values[0])
                elif i+14 <= (len(data_list) - 1) and data_list[i+13] != 0 and data_list[i+14] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (13 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+13])][column].values[0])
                elif i+15 <= (len(data_list) - 1) and data_list[i+14] != 0 and data_list[i+15] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+14])][column].values[0])
                elif i+16 <= (len(data_list) - 1) and data_list[i+15] != 0 and data_list[i+16] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+15])][column].values[0])
                elif i+17 <= (len(data_list) - 1) and data_list[i+16] != 0 and data_list[i+17] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+16])][column].values[0])
                elif i+18 <= (len(data_list) - 1) and data_list[i+17] != 0 and data_list[i+18] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+17])][column].values[0])
                elif i+19 <= (len(data_list) - 1) and data_list[i+18] != 0 and data_list[i+19] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+18])][column].values[0])
                elif i+20 <= (len(data_list) - 1) and data_list[i+19] != 0 and data_list[i+20] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+19])][column].values[0])
                elif i+21 <= (len(data_list) - 1) and data_list[i+20] != 0 and data_list[i+21] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+20])][column].values[0])                    
                elif i+22 <= (len(data_list) - 1) and data_list[i+21] != 0 and data_list[i+22] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+21])][column].values[0])                    
                elif i+23 <= (len(data_list) - 1) and data_list[i+22] != 0 and data_list[i+23] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+22])][column].values[0])                    
                elif i+24 <= (len(data_list) - 1) and data_list[i+23] != 0 and data_list[i+24] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+22]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (24 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+23])][column].values[0])                    
                elif i+25 <= (len(data_list) - 1) and data_list[i+24] != 0 and data_list[i+25] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+23]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+22]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (24 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (25 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+24])][column].values[0])                    
                elif i+26 <= (len(data_list) - 1) and data_list[i+25] != 0 and data_list[i+26] != 0:
                    rate_of_change = -1 * (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0] - df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+24]), column] = rate_of_change + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+23]), column] = (2 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+22]), column] = (3 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+21]), column] = (4 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+20]), column] = (5 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+19]), column] = (6 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+18]), column] = (7 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+17]), column] = (8 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+16]), column] = (9 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+15]), column] = (10 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+14]), column] = (11 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+13]), column] = (12 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+12]), column] = (14 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+11]), column] = (15 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+10]), column] = (16 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+9]), column] = (17 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+8]), column] = (18 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+7]), column] = (19 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+6]), column] = (20 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+5]), column] = (21 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+4]), column] = (22 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+3]), column] = (23 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+2]), column] = (24 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i+1]), column] = (25 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])                    
                    df.loc[(df['cow_code'] == c_code) & (df['year'] == years[i]), column] = (26 * rate_of_change) + (df[(df['cow_code'] == c_code) & (df['year'] == years[i+25])][column].values[0])                    
                else:
                    print(c_code, years[i], column)
                    fix_count+=1
                    
print(fix_count)

In [ ]:
df.loc[df['num_trade_states'] < 0, 'num_trade_states'] = 0
df.loc[df['prim_energy_consumption'] < 0, 'prim_energy_consumption'] = 0
df.loc[df['total_pop'] < 0, 'total_pop'] = 0
df.loc[df['urban_pop'] < 0, 'urban_pop'] = 0
df.loc[df['cinc_score'] < 0, 'cinc_score'] = 0
df.loc[df['military_personnel'] < 0, 'military_personnel'] = 0

df['export_import_ratio'] = df['export_dollars']/df['import_dollars']
df.drop(['export_dollars', 'import_dollars'], axis = 1, inplace = True)
df.drop(df[(df['num_trade_states'] == 0) & (df['export_dollars'] == 0) & (df['import_dollars'] == 0) & (df['export_dollars'] == 0) & (df['military_expenditure'] == 0) & (df['military_personnel'] == 0) & (df['prim_energy_consumption'] == 0)].index, inplace = True)

In [ ]:
print('total possible 0:\t', len(df))
print('num_trade_states == 0:\t', len(df[df['num_trade_states'] == 0]), '\t', pd.Series((len(df[df['num_trade_states'] == 0])/len(df))).values[0].round(3))
print('cinc_score == 0:\t', len(df[df['cinc_score'] == 0]), '\t', pd.Series((len(df[df['cinc_score'] == 0])/len(df))).values[0].round(3))
print('military_personnel == 0:\t', len(df[df['military_personnel'] == 0]), '\t', pd.Series((len(df[df['military_personnel'] == 0])/len(df))).values[0].round(3))
print('prim_energy_consumption == 0:\t', len(df[df['prim_energy_consumption'] == 0]), '\t', pd.Series((len(df[df['prim_energy_consumption'] == 0])/len(df))).values[0].round(3))
print('total_pop == 0:\t', len(df[df['total_pop'] == 0]), '\t', pd.Series((len(df[df['total_pop'] == 0])/len(df))).values[0].round(3))
print('urban_pop == 0:\t', len(df[df['urban_pop'] == 0]), '\t', pd.Series((len(df[df['urban_pop'] == 0])/len(df))).values[0].round(3))

# print('export_dollars == 0:\t', len(df[df['export_dollars'] == 0]), '\t', pd.Series((len(df[df['export_dollars'] == 0])/len(df))).values[0].round(3))
# print('import_dollars == 0:\t', len(df[df['import_dollars'] == 0]), '\t', pd.Series((len(df[df['import_dollars'] == 0])/len(df))).values[0].round(3))
# print('military_expenditure == 0:\t', len(df[df['military_expenditure'] == 0]), '\t', pd.Series((len(df[df['military_expenditure'] == 0])/len(df))).values[0].round(3))

In [ ]:
msno.matrix(df.drop(['cow_code', 'state_name', 'year'], axis = 1))

In [ ]:
df[df['num_trade_states'] == 0]

In [ ]:
df[df['prim_energy_consumption'] == 0]

In [ ]:
df.head()

In [ ]:
df.to_pickle('pickle/df_imputed.pkl')
df.to_csv('dataframe_exports/df_imputed.csv')

In [ ]:
df = pd.read_pickle('pickle/df_imputed.pkl')